In [ ]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.evaluation import MulticlassMetrics

# Read the dataset that was previously splitted due to jupyter's maximum file size issue
rawData = sc.textFile("*.csv")

def prepare(line):
    values = line.split(',')
    map(float, values)
    featureVector = Vectors.dense(values[:-1])
    # Decision tree labels varies from 0 to n-1
    label = float(values[-1])-1
    return LabeledPoint(label, featureVector)

data = rawData.map(lambda line : prepare(line))

trainData, cvData, testData = data.randomSplit([0.8, 0.1, 0.1])

trainData.cache()
cvData.cache()
testData.cache()

model = DecisionTree.trainClassifier(trainData, numClasses=7, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=4, maxBins=100)

predictions = model.predict(cvData.map(lambda x: x.features))
labelsAndPredictions = cvData.map(lambda lp: lp.label).zip(predictions) 
m = MulticlassMetrics(labelsAndPredictions)

# print m.confusionMatrix().toArray()
print m.precision()